## Importing necessary modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
from tqdm import tqdm # Cool progress bar

import random
import numpy as np
import pandas as pd
import sys
import os # read and manipulate local files

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import cv2
import seaborn as sns

from PIL import Image

import tensorflow.keras as ks
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K # F1-score metric

from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import   Lambda, Conv2D, MaxPool2D, UpSampling2D, BatchNormalization, Flatten
from tensorflow.keras.layers import  GlobalAveragePooling2D, Reshape, Multiply, Attention, add,Resizing,  Input, Dense
from tensorflow.keras.layers import Activation,AveragePooling2D, MaxPooling2D, Dropout, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential

# hide wornings
import warnings
warnings.filterwarnings('ignore')

# from keras import ops# For deeplab3

# import keras_cv # basnet

# from imgaug import augmenters as iaa ## for augmentation

## Defining paths

In [ ]:
model_name = 'Inception_resnetV2_Unet_616' #Basic_Unet'#'Vgg16_unet'

################################################################################################
## SETTING THE PATHS
PATH_TO_CODE ='/content/drive/MyDrive/TRIAL_v1/segmentation_task'
# PATH_TO_CODE =''
sys.path.append(PATH_TO_CODE)

# DATASET_PATH = r'C:\Users\ASUS\Desktop\segmentation & cropping\segmentation_task\segmentation_data43_resized_cropped_split'
DATASET_PATH = r'/content/drive/MyDrive/TRIAL_v1/segmentation_task/segmentation_data43_resized_cropped_split'
# DATASET_PATH = r'segmentation_data43_resized_cropped_split'
DATASET_PATH_TRAIN = os.path.join(DATASET_PATH, 'train')
DATASET_PATH_TEST = os.path.join(DATASET_PATH, 'test')


PATH_TO_SAVE_RESULT = os.path.join(PATH_TO_CODE, 'saved_outputs_segmentation_AUC', model_name)
PATH_BEST_SAVE_WEIGHT = os.path.join(PATH_TO_SAVE_RESULT,'saved_weights')
PATH_TO_SAVE_MODEL = os.path.join(PATH_TO_SAVE_RESULT, 'saved_models')
PATH_SAVE_HISTORY = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_training_history.csv')
PATH_SAVE_TIME = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_training_time.csv')
PATH_SAVE_AUGMENT_SAMPLE = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_augmented_sample.pdf')
PATH_SAVE_PIXEL_PERCENTAGE_PLOT = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_pixel_percentage.pdf')
# PATH_TO_SAVE_TUNER = os.path.join(PATH_TO_RESULT, 'saved_tuner_model')
################################################################################################

if not os.path.exists(PATH_TO_SAVE_RESULT):
 # If it does not exist, create it
    os.makedirs(PATH_TO_SAVE_RESULT)
################################################################################################
IMG_HEIGHT = 256
IMG_WIDTH = 256
ORIGINAL_IMAGE_SIZE = (IMG_HEIGHT, IMG_WIDTH)
COLOR_CHANNEL = 3

RESIZE_SHAPE = ORIGINAL_IMAGE_SIZE #(128, 128) #ORIGINAL_IMAGE_SIZE#(128, 128)# # #ORIGINAL_IMAGE_SIZE# (128, 128)#
MODEL_INPUT_SIZE = (RESIZE_SHAPE[0], RESIZE_SHAPE[1], COLOR_CHANNEL)

VALIDATION_SPLIT= 0.2
NUM_CLASSES = 2 # Disease and not disease

BATCH_SIZE = 10
EPOCHS = 35

INITIAL_LR = 0.001

TOTAL_DATA = 616
################################################################################################

################################################################################################

SAVE_RESULTS = True
SHOW_RESULTS = True

################################################################################################
# Setting the seed
SEED  = 123
RNG = np.random.default_rng(SEED) # Random number generator
tf.random.set_seed(SEED)

################################################################################################
# Checkpoint parameters val_binary_io_u
SCORE_TO_MONITOR = 'val_binary_io_u' # Score that checkpoints monitor during training
SCORE_OBJECTIVE  = 'max'          # 'max' or 'min', specifies whether the objective is to maximize the score or minimize it.
PATIENCE_LR_REDUCE = 5
MIN_LR = 1e-8
REDUCTION_FACTOR = 0.5            # Factor which lr will be reduced with at plateau
COOLDOWN_EPOCHS  = 2 #cooldown: Integer. Number of epochs to wait before resuming normal operation after the learning rate has been reduced.

INITIAL_LR = 0.001

## Defining performance metrics

Official segmentation metrics by keras: https://ks.io/api/metrics/segmentation_metrics/

In [ ]:
def f1_score(y_true, y_pred): # Dice coefficient
    """
    Calculate the F1 score, the harmonic mean of precision and recall, for binary classification.

    Args:
        y_true (Tensor): True binary labels.
        y_pred (Tensor): Predicted probabilities.

    Returns:
        float32: F1 score as a scalar.
    """
    # True Positives: round product of y_true and y_pred
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    # Actual Positives: round y_true
    P = K.sum(K.round(K.clip(y_true, 0, 1)))
    # Recall: TP / Actual Positives
    recall = TP / (P + K.epsilon())

    # Predicted Positives: round y_pred
    Pred_P = K.sum(K.round(K.clip(y_pred, 0, 1)))
    # Precision: TP / Predicted Positives
    precision = TP / (Pred_P + K.epsilon())

    # F1 Score: harmonic mean of precision and recall
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# source: https://www.tensorflow.org/api_docs/python/tf/keras/metrics/BinaryIoU
iou_score_binary = tf.keras.metrics.BinaryIoU(target_class_ids=[0, 1], threshold=0.5)

def accuracy_score(y_true, y_pred):
    """
    Calculate accuracy score between two binary masks.
    """
    correct = np.sum(y_true == y_pred)  # Count correct predictions
    total = y_true.size  # Total number of pixels
    return correct / total  # Accuracy calculation

def precision_score(groundtruth_mask, pred_mask):
    """
    Calculate precision score between two binary masks.
    """
    intersect = np.sum(pred_mask * groundtruth_mask)  # Calculate intersection
    total_pixel_pred = np.sum(pred_mask)  # Sum of predicted positives
    return np.mean(intersect / total_pixel_pred)  # Precision calculation

def recall_score(groundtruth_mask, pred_mask):
    """
    Calculate recall score between two binary masks.
    """
    intersect = np.sum(pred_mask * groundtruth_mask)  # Calculate intersection
    total_pixel_truth = np.sum(groundtruth_mask)  # Sum of actual positives
    return np.mean(intersect / total_pixel_truth)  # Recall calculation

## Reading the data

In [ ]:
# Function to read images from a directory using Pillow
def read_images_from_directory(directory_path):
    images = []
    for filename in sorted(os.listdir(directory_path)):
        # filename= filename.lower()

        if filename.endswith(('.png', '.jpg', '.JPG' )):  # Check for image file extensions
            img_path = os.path.join(directory_path, filename)
            img = Image.open(img_path)
            img_array = np.array(img)  # Convert the image to a numpy array if needed
            if img_array is not None:
                images.append(img_array)
    return images

# Paths to masks and original images within the dataset
masks_path_train = os.path.join(DATASET_PATH_TRAIN, 'data43_masks_binarised')
originals_path_train = os.path.join(DATASET_PATH_TRAIN, 'data43_original')

# Read images into variables
x_train = read_images_from_directory(originals_path_train)  # Original images
y_train = read_images_from_directory(masks_path_train)  # Masks

# Paths to masks and original images within the test dataset
masks_path_test = os.path.join(DATASET_PATH_TEST, 'data43_masks_binarised')
originals_path_test = os.path.join(DATASET_PATH_TEST, 'data43_original')

# Read images into variables
x_test = read_images_from_directory(originals_path_test)  # Original images
y_test = read_images_from_directory(masks_path_test)  # Masks

# Now x_train contains original images, and y_train contains mask images
print(f"Loaded {len(x_train)} original images into x_train")
print(f"Loaded {len(y_train)} mask images into y_train")

# Similarly, for test images
print(f"Loaded {len(x_test)} original images into x_test")
print(f"Loaded {len(y_test)} mask images into y_test")

Loaded 616 original images into x_train
Loaded 616 mask images into y_train
Loaded 23 original images into x_test
Loaded 23 mask images into y_test


## Preprocessing DO NOT RESIZE THE IMAGE-- IT GIVES FINE LINES IN THE OUTPUT

In [ ]:
# # # Resizing
# x_train = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in x_train]
# y_train = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in y_train]

# x_test = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in x_test]
# y_test = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in y_test]

# Convert the list to a NumPy array
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

# Expand mask dimensions to (batch, 256, 256, 1)
y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)

# Normalizing input between [0,1]
x_train = x_train.astype("float32")/ np.max(x_train)
x_test  = x_test.astype("float32")/np.max(x_test)

y_train = y_train.astype("float32")/ np.max(y_train)
y_test  = y_test.astype("float32")/np.max(y_test)

# Now x_train contains original images, and y_train contains mask images
print(f"Loaded {len(x_train)} original images into x_train")
print(f"Loaded {len(y_train)} mask images into y_train")

# Similarly, for test images
print(f"Loaded {len(x_test)} original images into x_test")
print(f"Loaded {len(y_test)} mask images into y_test")

Loaded 616 original images into x_train
Loaded 616 mask images into y_train
Loaded 23 original images into x_test
Loaded 23 mask images into y_test


## Essential functions

In [ ]:
def fit_and_save_best_model(model_name, model, epochs=EPOCHS):
    ##########################################################################################################
    saved_best_model_name = 'best_'+model_name+'.h5'

    model_saving_path = os.path.join(PATH_TO_SAVE_MODEL, saved_best_model_name)
    # Set up a model checkpoint to save the best model during training
    best_model_callback= ModelCheckpoint(model_saving_path,
                                          monitor=SCORE_TO_MONITOR,
                                          save_best_only=True,
                                          mode=SCORE_OBJECTIVE,
                                          verbose=1)

    reduce_LR = ReduceLROnPlateau(
        factor=REDUCTION_FACTOR,      # Factor by which the learning rate will be reduced. new_lr = lr * factor
        patience=PATIENCE_LR_REDUCE, # original was 5      # Number of epochs with no improvement after which learning rate will be reduced.
        verbose=1,       # int. 0: quiet, 1: update messages.
        min_lr=MIN_LR   # Lower bound on the learning rate.
    )

    my_callbacks = [best_model_callback , reduce_LR]
    ##########################################################################################################

    start_time = time.time()

    # Fitting the model
    train_history = model.fit(
        x_train,
        y_train,
        epochs= epochs,
        batch_size=BATCH_SIZE,
        validation_split= VALIDATION_SPLIT,
        callbacks=my_callbacks,
    )
    total_time = time.time() - start_time

    return saved_best_model_name, train_history, total_time, model

## Models
### Models worked
- Model01: Basic basic_unet
- Model02: unet_vgg16 (my own custom)
- Model03: TransUNet
- Model04: DeepLab-v3Plus
- Model05: ResUNet
- Model06: UNet_with_attention
- Model07: Multi-resUnet
- Model08: Inception_resnetV2 (my own custom)



### InceptionResNetV2---Works--checked with batch size10

In [ ]:
from tensorflow.keras.applications import  InceptionResNetV2
# def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
#     """Function to add 2 convolutional layers with the parameters passed to it"""
#     # first layer
#     x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
#               kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
#     if batchnorm:
#         x = BatchNormalization()(x)
#     x = Activation('relu')(x)

#     # second layer
#     x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
#               kernel_initializer = 'he_normal', padding = 'same')(x)
#     if batchnorm:
#         x = BatchNormalization()(x)
#     x = Activation('relu')(x)

#     return x

# def get_unet_InceptionResNetV2(n_filters=16, batchnorm=True, dropout=0.1):
#     InceptionResNetV2_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=MODEL_INPUT_SIZE)

#     # Encoder
#     encoder = InceptionResNetV2_base.get_layer('activation_89').output

#     # Bottleneck
#     bottleneck = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)

#     # Resizing for skip connections
#     resized_skip1 = tf.image.resize(InceptionResNetV2_base.get_layer('activation_89').output, [28, 28])
#     resized_skip2 = tf.image.resize(InceptionResNetV2_base.get_layer('max_pooling2d').output, [56, 56])
#     resized_skip3 = tf.image.resize(InceptionResNetV2_base.get_layer('conv2d_1').output, [112, 112])

#     # Decoder
#     u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides=(2, 2), padding='same')(bottleneck)
#     u6 = Concatenate(axis=-1)([u6, resized_skip1])
#     u6 = Dropout(dropout)(u6)
#     c6 = conv2d_block(u6, n_filters * 8, kernel_size=3, batchnorm=batchnorm)

#     u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides=(2, 2), padding='same')(c6)
#     u7 = Concatenate(axis=-1)([u7, resized_skip2])
#     u7 = Dropout(dropout)(u7)
#     c7 = conv2d_block(u7, n_filters * 4, kernel_size=3, batchnorm=batchnorm)

#     u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides=(2, 2), padding='same')(c7)

#     u8 = Dropout(dropout)(u8)
#     c8 = conv2d_block(u8, n_filters * 2, kernel_size=3, batchnorm=batchnorm)

#     # Final upsampling to match the original input size
#     u9 = Conv2DTranspose(n_filters, (3, 3), strides=(2, 2), padding='same')(c8)
#     u9 = Dropout(dropout)(u9)
#     c9 = conv2d_block(u9, n_filters, kernel_size=3, batchnorm=batchnorm)

#     # Additional resizing layer to ensure output is 256x256
#     final_output = Resizing(256, 256, interpolation='bilinear')(c9)

#     outputs = Conv2D(NUM_CLASSES-1, (1, 1), activation='sigmoid')(final_output)
#     model = Model(inputs=InceptionResNetV2_base.input, outputs=[outputs])
#     optimizer = Adam(learning_rate=INITIAL_LR)
#     model.compile(loss='binary_crossentropy',
#                   metrics=['accuracy', f1_score, iou_score_binary],
#                   optimizer=optimizer)

#     return model

# def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
#     """Function to add 2 convolutional layers with the parameters passed to it"""
#     # first layer
#     x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
#               kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
#     if batchnorm:
#         x = BatchNormalization()(x)
#     x = Activation('relu')(x)

#     # second layer
#     x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
#               kernel_initializer = 'he_normal', padding = 'same')(x)
#     if batchnorm:
#         x = BatchNormalization()(x)
#     x = Activation('relu')(x)

#     return x

# def get_unet_InceptionResNetV2(n_filters=16, batchnorm=True, dropout=0.1):
#     InceptionResNetV2_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=MODEL_INPUT_SIZE)

#     # Encoder
#     encoder = InceptionResNetV2_base.get_layer('activation_89').output

#     # Bottleneck
#     bottleneck = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)

#     # Resizing for skip connections using Keras Resizing layer
#     # Use a Resizing layer for each skip connection to correctly handle KerasTensors
#     resized_skip1 = Resizing(28, 28, interpolation='bilinear')(InceptionResNetV2_base.get_layer('activation_89').output)
#     resized_skip2 = Resizing(56, 56, interpolation='bilinear')(InceptionResNetV2_base.get_layer('max_pooling2d').output)
#     resized_skip3 = Resizing(112, 112, interpolation='bilinear')(InceptionResNetV2_base.get_layer('conv2d_1').output)


#     # Decoder
#     u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides=(2, 2), padding='same')(bottleneck)
#     # Check the shape of u6 before concatenating to ensure it matches the resized_skip1 shape
#     # print(f"Shape of u6 before concatenation with resized_skip1: {u6.shape}")
#     # print(f"Shape of resized_skip1: {resized_skip1.shape}")
#     u6 = Concatenate(axis=-1)([u6, resized_skip1])
#     u6 = Dropout(dropout)(u6)
#     c6 = conv2d_block(u6, n_filters * 8, kernel_size=3, batchnorm=batchnorm)

#     u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides=(2, 2), padding='same')(c6)
#     # Check the shape of u7 before concatenating to ensure it matches the resized_skip2 shape
#     # print(f"Shape of u7 before concatenation with resized_skip2: {u7.shape}")
#     # print(f"Shape of resized_skip2: {resized_skip2.shape}")
#     u7 = Concatenate(axis=-1)([u7, resized_skip2])
#     u7 = Dropout(dropout)(u7)
#     c7 = conv2d_block(u7, n_filters * 4, kernel_size=3, batchnorm=batchnorm)

#     u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides=(2, 2), padding='same')(c7)
#     # There is no concatenation with resized_skip3 here. If resized_skip3 was intended for u8,
#     # you would concatenate it here. Based on the original code structure, it seems
#     # resized_skip3 might have been a leftover or intended for a different part of the decoder.
#     # If you intended to use it, uncomment and adapt the following line:
#     # u8 = Concatenate(axis=-1)([u8, resized_skip3])

#     u8 = Dropout(dropout)(u8)
#     c8 = conv2d_block(u8, n_filters * 2, kernel_size=3, batchnorm=batchnorm)

#     # Final upsampling to match the original input size
#     u9 = Conv2DTranspose(n_filters, (3, 3), strides=(2, 2), padding='same')(c8)
#     u9 = Dropout(dropout)(u9)
#     c9 = conv2d_block(u9, n_filters, kernel_size=3, batchnorm=batchnorm)

#     # Additional resizing layer to ensure output is 256x256
#     # This resizing layer seems redundant if the upsampling layers are already
#     # designed to reach the target size. Double check the expected output size after u9/c9.
#     # If the previous layers correctly bring the size to 256x256, you can remove this layer.
#     # If not, keep it to ensure the final output shape.
#     final_output = Resizing(256, 256, interpolation='bilinear')(c9)

#     outputs = Conv2D(NUM_CLASSES-1, (1, 1), activation='sigmoid')(final_output)
#     model = Model(inputs=InceptionResNetV2_base.input, outputs=[outputs])
#     optimizer = Adam(learning_rate=INITIAL_LR)
#     model.compile(loss='binary_crossentropy',
#                   metrics=['accuracy', f1_score, iou_score_binary],
#                   optimizer=optimizer)

#     return model

# Conv block function
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size),
               kernel_initializer='he_normal', padding='same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size),
               kernel_initializer='he_normal', padding='same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x

# Model function
def get_unet_InceptionResNetV2(n_filters=16, batchnorm=True, dropout=0.1):
    InceptionResNetV2_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=MODEL_INPUT_SIZE)

    # Encoder
    encoder = InceptionResNetV2_base.get_layer('activation_89').output

    # Bottleneck
    bottleneck = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)

    # Resizing skip connections with Lambda
    resized_skip1 = Lambda(lambda x: tf.image.resize(x, [28, 28]))(
        InceptionResNetV2_base.get_layer('activation_89').output)
    resized_skip2 = Lambda(lambda x: tf.image.resize(x, [56, 56]))(
        InceptionResNetV2_base.get_layer('max_pooling2d').output)
    resized_skip3 = Lambda(lambda x: tf.image.resize(x, [112, 112]))(
        InceptionResNetV2_base.get_layer('conv2d_1').output)

    # Decoder
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides=(2, 2), padding='same')(bottleneck)
    u6 = Concatenate(axis=-1)([u6, resized_skip1])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides=(2, 2), padding='same')(c6)
    u7 = Concatenate(axis=-1)([u7, resized_skip2])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides=(2, 2), padding='same')(c7)
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters, (3, 3), strides=(2, 2), padding='same')(c8)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters, kernel_size=3, batchnorm=batchnorm)

    # Resize to match final output size
    final_output = Resizing(256, 256, interpolation='bilinear')(c9)

    outputs = Conv2D(NUM_CLASSES - 1, (1, 1), activation='sigmoid')(final_output)

    model = Model(inputs=InceptionResNetV2_base.input, outputs=[outputs])
    optimizer = Adam(learning_rate=INITIAL_LR)
    model.compile(loss='binary_crossentropy',
                  metrics=['accuracy', f1_score, iou_score_binary],
                  optimizer=optimizer)

    return model

## Train and Results

In [ ]:
# model= get_basic_unet() ## Model1
# model= get_unet_vgg16() ## Model2
# model= get_TransUNet() ## Model3
# model= get_DeepLabV3Plus() ## Model4
# model = get_ResUNet_v3() ## Model5
# model = UNet_with_attention() ## Model6
# model = get_multiresunet() ## Model7
model = get_unet_InceptionResNetV2() ## Model8

model.summary()

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 127, 127,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 127, 127,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 127, 127,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 125, 125,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 125, 125,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 125, 125,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 125, 125,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 125, 125,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 125, 125,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 62, 62,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 62, 62,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 62, 62,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 62, 62,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 60, 60,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 60,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 60, 60,    │          0 │ batch_normalizat

 Total params: 9,719,841 (37.08 MB)

 Trainable params: 9,705,377 (37.02 MB)

 Non-trainable params: 14,464 (56.50 KB)

In [ ]:
# training the model and saving the best model as a check point
best_model_name, train_history, total_time, model = fit_and_save_best_model(model_name, model)

Epoch 1/35
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 843ms/step - accuracy: 0.3431 - binary_io_u: 0.1810 - f1_score: 0.0567 - loss: 0.8284
Epoch 1: val_binary_io_u improved from -inf to 0.47729, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Inception_resnetV2_Unet_616/saved_models/best_Inception_resnetV2_Unet_616.h5


50/50 ━━━━━━━━━━━━━━━━━━━━ 154s 1s/step - accuracy: 0.3491 - binary_io_u: 0.1842 - f1_score: 0.0562 - loss: 0.8264 - val_accuracy: 0.9545 - val_binary_io_u: 0.4773 - val_f1_score: 1.2462e-04 - val_loss: 72.9462 - learning_rate: 0.0010
Epoch 2/35
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.9545 - binary_io_u: 0.4775 - f1_score: 9.3684e-04 - loss: 0.6604
Epoch 2: val_binary_io_u did not improve from 0.47729
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - accuracy: 0.9543 - binary_io_u: 0.4774 - f1_score: 0.0010 - loss: 0.6600 - val_accuracy: 0.9539 - val_binary_io_u: 0.4770 - val_f1_score: 3.4723e-04 - val_loss: 0.7408 - learning_rate: 0.0010
Epoch 3/35
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.9490 - binary_io_u: 0.4774 - f1_score: 0.0116 - loss: 0.5815
Epoch 3: val_binary_io_u did not improve from 0.47729
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - accuracy: 0.9489 - binary_io_u: 0.4773 - f1_score: 0.0115 - loss: 0.5792 - val_accuracy: 0.6591 - val_binary_io_u: 0.3718 - v

50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 115ms/step - accuracy: 0.9502 - binary_io_u: 0.4816 - f1_score: 0.0240 - loss: 0.3253 - val_accuracy: 0.9481 - val_binary_io_u: 0.6450 - val_f1_score: 0.4622 - val_loss: 0.5686 - learning_rate: 0.0010
Epoch 5/35
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.9678 - binary_io_u: 0.6810 - f1_score: 0.5596 - loss: 0.2367
Epoch 5: val_binary_io_u improved from 0.64503 to 0.69726, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Inception_resnetV2_Unet_616/saved_models/best_Inception_resnetV2_Unet_616.h5


50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 103ms/step - accuracy: 0.9677 - binary_io_u: 0.6806 - f1_score: 0.5586 - loss: 0.2364 - val_accuracy: 0.9706 - val_binary_io_u: 0.6973 - val_f1_score: 0.5000 - val_loss: 0.4832 - learning_rate: 0.0010
Epoch 6/35
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.9712 - binary_io_u: 0.7208 - f1_score: 0.6348 - loss: 0.1904
Epoch 6: val_binary_io_u did not improve from 0.69726
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - accuracy: 0.9712 - binary_io_u: 0.7206 - f1_score: 0.6344 - loss: 0.1901 - val_accuracy: 0.9664 - val_binary_io_u: 0.6182 - val_f1_score: 0.3393 - val_loss: 0.3121 - learning_rate: 0.0010
Epoch 7/35
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.9741 - binary_io_u: 0.7445 - f1_score: 0.6743 - loss: 0.1517
Epoch 7: val_binary_io_u improved from 0.69726 to 0.75030, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Inception_resnetV2_Unet_616/saved_models/best_Inception_resnetV2_Unet_616.h5


50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.9740 - binary_io_u: 0.7441 - f1_score: 0.6736 - loss: 0.1516 - val_accuracy: 0.9748 - val_binary_io_u: 0.7503 - val_f1_score: 0.6182 - val_loss: 0.1509 - learning_rate: 0.0010
Epoch 8/35
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.9754 - binary_io_u: 0.7557 - f1_score: 0.6917 - loss: 0.1281
Epoch 8: val_binary_io_u improved from 0.75030 to 0.75683, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Inception_resnetV2_Unet_616/saved_models/best_Inception_resnetV2_Unet_616.h5


...

50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 110ms/step - accuracy: 0.9853 - binary_io_u: 0.8421 - f1_score: 0.8124 - loss: 0.0470 - val_accuracy: 0.9787 - val_binary_io_u: 0.7780 - val_f1_score: 0.6708 - val_loss: 0.0646 - learning_rate: 5.0000e-04
Epoch 23/35
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.9855 - binary_io_u: 0.8453 - f1_score: 0.8168 - loss: 0.0458
Epoch 23: val_binary_io_u improved from 0.77804 to 0.78072, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Inception_resnetV2_Unet_616/saved_models/best_Inception_resnetV2_Unet_616.h5


50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 106ms/step - accuracy: 0.9855 - binary_io_u: 0.8454 - f1_score: 0.8168 - loss: 0.0458 - val_accuracy: 0.9780 - val_binary_io_u: 0.7807 - val_f1_score: 0.6798 - val_loss: 0.0664 - learning_rate: 5.0000e-04
Epoch 24/35
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.9855 - binary_io_u: 0.8448 - f1_score: 0.8157 - loss: 0.0457
Epoch 24: val_binary_io_u improved from 0.78072 to 0.78997, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Inception_resnetV2_Unet_616/saved_models/best_Inception_resnetV2_Unet_616.h5


50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.9855 - binary_io_u: 0.8449 - f1_score: 0.8157 - loss: 0.0457 - val_accuracy: 0.9786 - val_binary_io_u: 0.7900 - val_f1_score: 0.6895 - val_loss: 0.0669 - learning_rate: 5.0000e-04
Epoch 25/35
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.9855 - binary_io_u: 0.8450 - f1_score: 0.8165 - loss: 0.0449
Epoch 25: val_binary_io_u did not improve from 0.78997
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.9855 - binary_io_u: 0.8451 - f1_score: 0.8165 - loss: 0.0450 - val_accuracy: 0.9792 - val_binary_io_u: 0.7842 - val_f1_score: 0.6784 - val_loss: 0.0654 - learning_rate: 5.0000e-04
Epoch 26/35
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.9862 - binary_io_u: 0.8492 - f1_score: 0.8211 - loss: 0.0432
Epoch 26: val_binary_io_u did not improve from 0.78997
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - accuracy: 0.9861 - binary_io_u: 0.8493 - f1_score: 0.8212 - loss: 0.0433 - val_accuracy: 0.9789 - val_binary_io_u: 0.7882 - v

50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.9857 - binary_io_u: 0.8466 - f1_score: 0.8174 - loss: 0.0429 - val_accuracy: 0.9782 - val_binary_io_u: 0.7911 - val_f1_score: 0.7002 - val_loss: 0.0658 - learning_rate: 5.0000e-04
Epoch 29/35
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.9859 - binary_io_u: 0.8493 - f1_score: 0.8212 - loss: 0.0421
Epoch 29: val_binary_io_u improved from 0.79113 to 0.79371, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Inception_resnetV2_Unet_616/saved_models/best_Inception_resnetV2_Unet_616.h5


50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 104ms/step - accuracy: 0.9859 - binary_io_u: 0.8495 - f1_score: 0.8212 - loss: 0.0422 - val_accuracy: 0.9794 - val_binary_io_u: 0.7937 - val_f1_score: 0.6969 - val_loss: 0.0654 - learning_rate: 5.0000e-04
Epoch 30/35
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.9864 - binary_io_u: 0.8524 - f1_score: 0.8258 - loss: 0.0410
Epoch 30: val_binary_io_u did not improve from 0.79371
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.9864 - binary_io_u: 0.8525 - f1_score: 0.8258 - loss: 0.0410 - val_accuracy: 0.9772 - val_binary_io_u: 0.7857 - val_f1_score: 0.6937 - val_loss: 0.0695 - learning_rate: 5.0000e-04
Epoch 31/35
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.9865 - binary_io_u: 0.8553 - f1_score: 0.8298 - loss: 0.0402
Epoch 31: val_binary_io_u did not improve from 0.79371

Epoch 31: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.9865 - binary_io_u: 0.8555 - f1_sco

## Loading the best model

In [ ]:
# ##Now, load the best model
# best_model = ks.models.load_model(os.path.join(PATH_TO_SAVE_MODEL, best_model_name),
#                                   custom_objects={'f1_score': f1_score,
#                                                   'binary_io_u':iou_score_binary,
#                                                  })

# ##Now, load the best model
# best_model = ks.models.load_model("/content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation/616/Inception_resnetV2_Unet_616/saved_models/best_Inception_resnetV2_Unet_616.h5",
#                                   custom_objects={'f1_score': f1_score,
#                                                   'binary_io_u':iou_score_binary,
#                                                  })
# best_model =model custom_scale_lay… │  "CustomScaleLayer":CustomScaleLayer

best_model=model

## Visualize model predictions

In [ ]:
# Get predictions from the model
predictions = best_model.predict(x_test, verbose=1)

USER_DETERMINED_THRESHOLD = 0.5
thresholded_predictions  = (predictions  >= USER_DETERMINED_THRESHOLD)
# Removing the color channel
thresholded_predictions_without_color_channel = np.squeeze(thresholded_predictions, axis=-1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step


# AUC

In [ ]:
from sklearn.metrics import roc_auc_score

# Flatten masks for AUC computation
y_true_flat = y_test.flatten()
y_score_flat = predictions.flatten()  # Predicted probabilities

In [ ]:
# Bootstrapping AUC
n_iterations = 100
rng = np.random.default_rng(SEED)
bootstrap_aucs = []

for _ in range(n_iterations):
    indices = rng.integers(0, len(y_true_flat), len(y_true_flat))
    y_true_sample = y_true_flat[indices]
    y_score_sample = y_score_flat[indices]

    # Ensure both classes are present in the sample
    if len(np.unique(y_true_sample)) < 2:
        continue

    auc = roc_auc_score(y_true_sample, y_score_sample)
    bootstrap_aucs.append(auc)

In [ ]:
# Calculate statistics
mean_auc = np.mean(bootstrap_aucs)
std_auc = np.std(bootstrap_aucs)
lower_ci = np.percentile(bootstrap_aucs, 2.5)
upper_ci = np.percentile(bootstrap_aucs, 97.5)

# Print AUC statistics
print(f"Bootstrapped AUC: Mean={mean_auc:.4f}, Std={std_auc:.4f}, 95% CI=({lower_ci:.4f}, {upper_ci:.4f})")

# Optional saving
if SAVE_RESULTS:
    auc_df = pd.DataFrame({'AUC Scores': bootstrap_aucs})
    auc_df.to_csv(os.path.join(PATH_TO_SAVE_RESULT, f"{model_name}_bootstrapped_auc.csv"), index=False)

Bootstrapped AUC: Mean=0.9697, Std=0.0005, 95% CI=(0.9686, 0.9705)
